## Probability

In [ ]:
# The %... is an iPython thing, and is not part of the Python language.
# In this case we're just telling the plotting library to draw things on
# the notebook, instead of on a separate window.
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
plt.rc('figure', figsize=(8,5))

### Simulate Coin toss

In [ ]:
def throw_a_coin(N): # Use this function a simulate coin toss
    return np.random.choice(['H','T'], size=N)  # np.random.choice choose from [H, T] with replacement N times

In [ ]:
throws=throw_a_coin(40)
print "Throws:", throws
print "Number of Heads:", np.sum(throws=='H')
print "p1 = Number of Heads/Total Throws:", np.sum(throws=='H')/40.

Let's do more trials:

In [ ]:
trials=[10, 20, 50, 70, 100, 200, 500, 800, 1000, 2000, 5000, 7000, 10000]
plt.plot(trials, [np.sum(throw_a_coin(j)=='H')/np.float(j) for j in trials], 'o-', alpha=0.6);
plt.xscale("log")
plt.axhline(0.5, 0, 1, color='r');
plt.xlabel('number of trials');
plt.ylabel('probability of heads from simulation');
plt.title('frequentist probability of heads');

### Bernoulli Random Variables (in scipy.stats)

In [ ]:
from scipy.stats import bernoulli
#bernoulli random variable
brv=bernoulli(p=0.3)
brv.rvs(size=20)

A complete list of this bernoilli random variable object:
```python

rvs(p, loc=0, size=1, random_state=None)	Random variates.
pmf(x, p, loc=0)	Probability mass function.
logpmf(x, p, loc=0)	Log of the probability mass function.
cdf(x, p, loc=0)	Cumulative distribution function.
logcdf(x, p, loc=0)	Log of the cumulative distribution function.
sf(x, p, loc=0)	Survival function (also defined as 1 - cdf, but sf is sometimes more accurate).
logsf(x, p, loc=0)	Log of the survival function.
ppf(q, p, loc=0)	Percent point function (inverse of cdf — percentiles).
isf(q, p, loc=0)	Inverse survival function (inverse of sf).
stats(p, loc=0, moments='mv')	Mean(‘m’), variance(‘v’), skew(‘s’), and/or kurtosis(‘k’).
entropy(p, loc=0)	(Differential) entropy of the RV.
expect(func, args=(p,), loc=0, lb=None, ub=None, conditional=False)	Expected value of a function (of one argument) with respect to the distribution.
median(p, loc=0)	Median of the distribution.
mean(p, loc=0)	Mean of the distribution.
var(p, loc=0)	Variance of the distribution.
std(p, loc=0)	Standard deviation of the distribution.
interval(alpha, p, loc=0)	Endpoints of the range that contains alpha percent of the distribution
```

In [ ]:
bernoulli.stats(0.5, loc=2, moments='m')

* We can have a look at the distribution of the variates generated

In [ ]:
a=brv.rvs(size=100)
weights = np.ones_like(a)/float(len(a))
plt.hist(a, weights=weights)
# hist function doesn't provide automatic normalizing frequency to percentage, but you can achieve it by setting weights
# It's a trick I found on Stackoverflow
plt.xlim(-1,2)
plt.ylim(0,1)
plt.show()

* Another way of visualizing pmf, use bar chart.

In [ ]:
event_space=[0,1]
plt.figure(figsize=(12,8))
colors=sns.color_palette()

for i, p in enumerate([0.1, 0.2, 0.5, 0.7]):
    ax = plt.subplot(1, 4, i+1)
    plt.bar(event_space, bernoulli.pmf(event_space, p), label=p, color=colors[i], alpha=0.5)
    plt.plot(event_space, bernoulli.cdf(event_space, p), color=colors[i], alpha=0.5)

    ax.xaxis.set_ticks(event_space)
   
    plt.ylim((0,1))
    plt.legend(loc=0)
    if i == 0:
        plt.ylabel("PDF at $k$")
plt.tight_layout()

### Uniform Distibution

In [ ]:
from scipy.stats import uniform
r=uniform.rvs(size=1000) #generates uniformly distributed variables in [0,1]

In [ ]:
plt.hist(r, histtype='stepfilled', alpha=0.8)
plt.show()

### Binomial distribution
has two parameters n and p

In [ ]:
from scipy.stats import binom
plt.figure(figsize=(12,6))
k = np.arange(0, 200)
for p, color in zip([0.1, 0.3, 0.7, 0.7, 0.9], colors):
    rv = binom(200, p)
    plt.plot(k, rv.pmf(k), '.', lw=2, color=color, label=p)
    plt.fill_between(k, rv.pmf(k), color=color, alpha=0.5)
q=plt.legend()
plt.title("Binomial distribution")
plt.tight_layout()
q=plt.ylabel("PDF at $k$")
q=plt.xlabel("$k$")

### The various ways to get random numbers
* ``np.random.choice`` chooses items randomly from an array, with or without replacement
* ``np.random.random`` gives us uniform randoms on [0.0,1.0)
* ``np.random.randint`` gives us random integers in some range.
* ``np.random.randn`` gives us random samples from a Normal distribution.
* ``scipy.stats.distrib`` gives us stuff from a distribution. Here distrib could be binom for example, as above. distrib.pdf or distrib.pmf give us the density or mass function, while cdf gives us the cumulaive distribution function. Just using distrib as a function with its params creates a random variable generating object, from which random variables can be generated in the form distrib(params).rvs(size).

In [ ]:
from scipy.stats import expon
xpts=np.arange(-2,3,0.1)
plt.plot(xpts,expon.pdf(xpts, scale=1./2.),'o')
plt.hist(expon.rvs(size=1000, scale=1./2.), normed=True, alpha=0.5, bins=30);
plt.xlabel("x")
plt.title("exponential pdf and samples(normalized)");

## Perform hypothesis testing
### example: z-test
 Many frequentist methods for hypothesis testing roughly involve the following steps:

1. Writing down the hypotheses, notably the **null hypothesis** which is the *opposite* of the hypothesis you want to prove (with a certain degree of confidence).
2. Computing a **test statistics**, a mathematical formula depending on the test type, the model, the hypotheses, and the data.
3. Using the computed value to accept the hypothesis, reject it, or fail to conclude.
  
Here, we flip a coin $n$ times and we observe $h$ heads. We want to know whether the coin is fair (null hypothesis). This example is extremely simple yet quite good for pedagogical purposes. 

* Let's suppose that, after $n=100$ flips, we get $h=61$ heads. We choose a significance level of 0.05: is the coin fair or not? Our null hypothesis is: *the coin is fair* ($q = 1/2$).

In [ ]:
n = 100  # number of coin flips
h = 61  # number of heads
q = .5  # null-hypothesis of fair coin

xbar = float(h)/n #sample mean

# Compute the z-score
z = (xbar - q) * np.sqrt(n / (q*(1-q)));
print 'z=', z

# Compute p-value
pval = 2 * (1 - sp.stats.norm.cdf(z)); pval
print 'pval=', pval

# Conclude
print("Null hypothesis is rejected" if pval < 0.05 else "Null hypothesis is not rejected")

## Bootstrap
Bootstrap tries to approximate our sampling distribution. If we knew the true parameters of the population, we could generate M fake datasets. Then we could compute the parameter (or another estimator) on each one of these, to get a empirical sampling distribution of the parameter or estimator, and which will give us an idea of how typical our sample is, and thus, how good our parameter estimations from our sample are. (again from murphy)

But we dont have the true parameter. So we generate these samples, using the parameter we calculated. Or, alteratively, we sample with replacement the X from our original sample D, generating many fake datasets, and then compute the distribution on the parameters as before.

We have data set of the record of new-born babies:

Forty-four babies -- a new record -- were born in one 24-hour period at the Mater Mothers' Hospital in Brisbane, Queensland, Australia, on December 18, 1997. For each of the 44 babies, The Sunday Mail recorded the time of birth, the sex of the child, and the birth weight in grams. Also included is the number of minutes since midnight for each birth.


In [ ]:
df = pd.read_table("babyboom.dat.txt", header=None, sep='\s+', 
                   names=['24hrtime','sex','weight','minutes'])
df.head()

We know it may be well-fitted in a poisson distribution. In this case, the time difference of new-born babies may be well modeled by a exponential ditribution

In [ ]:
timediffs = df.minutes.diff()[1:] #extract the time difference
print timediffs
timediffs.hist(bins=20);

### Non-parametric Bootstrap
Then we do bootstrapping to estimate the average time difference of birth

In [ ]:
# Number of samples
M_samples=10000

# The size of each sample
N_points = timediffs.shape[0]

#Use np.random.choice to do bootstrap
bs_np = np.random.choice(timediffs, size=(M_samples, N_points))

#Calculate statistics
sd_mean=np.mean(bs_np, axis=1)
sd_std=np.std(bs_np, axis=1)

plt.hist(sd_mean, bins=30, normed=True, alpha=0.5,label="samples");

# Do a kernel density estimation of the sampled means (smooth)
sns.kdeplot(sd_mean, label="inferred distribution")
plt.axvline(timediffs.mean(), 0, 1, color='r', label='Our Sample')
plt.legend()

### Parametric bootstrap
We can also do it in a parametric way. We first estimate lambda from the sample mean, and then generate many samples from the exponential distribution that's defined by the lambda. 

In [ ]:
lambda_from_mean = 1./timediffs.mean()
rv = expon(scale=1./lambda_from_mean)
M_samples=10000
N_points = timediffs.shape[0]
bs_p = rv.rvs(size=(M_samples, N_points))
sd_mean_p=np.mean(bs_p, axis=1)
sd_std_p=np.std(bs_p, axis=1)
plt.hist(sd_mean_p, bins=30, normed=True, alpha=0.5);
sns.kdeplot(sd_mean_p);
plt.axvline(timediffs.mean(), 0, 1, color='r', label='Our Sample')